In [12]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient("localhost", 27017)
client.drop_database("bookRegistry")
client.list_database_names()


['admin', 'config', 'local']

In [13]:
db = client.bookRegistry
db.list_collection_names()


[]

In [14]:
authors = db.authors.insert_many(
    [{"name": "Max Schwarz", "age": 29}, {"name": "Manuel Lor", "age": 30}]
).inserted_ids
for x in db.authors.find():
    pprint(x)


{'_id': ObjectId('644fffa18b7e8cd7435c3434'), 'age': 29, 'name': 'Max Schwarz'}
{'_id': ObjectId('644fffa18b7e8cd7435c3435'), 'age': 30, 'name': 'Manuel Lor'}


In [15]:
db.books.insert_one({"name": "My Favorite Book", "authors": authors})
db.books.find_one()


{'_id': ObjectId('644fffa18b7e8cd7435c3436'),
 'name': 'My Favorite Book',
 'authors': [ObjectId('644fffa18b7e8cd7435c3434'),
  ObjectId('644fffa18b7e8cd7435c3435')]}

In [16]:
creators = db.books.aggregate(
    [
        {
            "$lookup": {
                "from": "authors",
                "localField": "authors",
                "foreignField": "_id",
                "as": "creators",
            }
        }
    ]
)
for x in creators:
    pprint(x)


{'_id': ObjectId('644fffa18b7e8cd7435c3436'),
 'authors': [ObjectId('644fffa18b7e8cd7435c3434'),
             ObjectId('644fffa18b7e8cd7435c3435')],
 'creators': [{'_id': ObjectId('644fffa18b7e8cd7435c3434'),
               'age': 29,
               'name': 'Max Schwarz'},
              {'_id': ObjectId('644fffa18b7e8cd7435c3435'),
               'age': 30,
               'name': 'Manuel Lor'}],
 'name': 'My Favorite Book'}


In [17]:
client.close()
